# Unit 6: Outlier Detection and Treatment - Exercises

## Instructions

This notebook contains exercises to test your understanding of outlier detection and treatment. Each exercise has a specific point value, totaling **110 points**.

**Grading:**
- 90-110 points: Excellent ✅
- 70-89 points: Good 👍
- 50-69 points: Needs improvement ⚠️
- Below 50: Review the theory and practical notebooks 📚

**Tips:**
- Read each question carefully
- Test your code in separate cells before finalizing
- Comment your code for better understanding
- Document your decisions

---

## Setup

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Load datasets
diamonds = sns.load_dataset('diamonds')
mpg = sns.load_dataset('mpg')

print("✅ Setup complete!")
print(f"Diamonds shape: {diamonds.shape}")
print(f"MPG shape: {mpg.shape}")

---

## Exercise 1: Visual Outlier Detection (20 points)

Create comprehensive visualizations to identify outliers in the `price` variable of the diamonds dataset:

**Tasks:**
1. Create a box plot (5 points)
2. Create a histogram (5 points)
3. Create a scatter plot (index vs price) (5 points)
4. Identify visual outliers and describe what you see (5 points)

**Expected output:**
- 3 subplots in one figure
- Properly labeled axes and titles
- Written interpretation

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - Exercise 1
print("VISUAL OUTLIER DETECTION: Diamond Prices")
print("="*80)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Box plot
bp = axes[0].boxplot(diamonds['price'], vert=True, patch_artist=True,
                      boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0].set_ylabel('Price ($)', fontsize=11)
axes[0].set_title('Box Plot: Diamond Prices', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add quartile annotations
Q1 = diamonds['price'].quantile(0.25)
Q2 = diamonds['price'].quantile(0.50)
Q3 = diamonds['price'].quantile(0.75)
axes[0].text(1.25, Q1, f'Q1: ${Q1:,.0f}', verticalalignment='center', fontsize=9)
axes[0].text(1.25, Q2, f'Q2: ${Q2:,.0f}', verticalalignment='center', fontsize=9)
axes[0].text(1.25, Q3, f'Q3: ${Q3:,.0f}', verticalalignment='center', fontsize=9)

# 2. Histogram
axes[1].hist(diamonds['price'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(diamonds['price'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: ${diamonds["price"].mean():,.0f}')
axes[1].axvline(diamonds['price'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Median: ${diamonds["price"].median():,.0f}')
axes[1].set_xlabel('Price ($)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Histogram: Diamond Prices', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# 3. Scatter plot
axes[2].scatter(range(len(diamonds)), diamonds['price'], alpha=0.4, s=10)
axes[2].set_xlabel('Index', fontsize=11)
axes[2].set_ylabel('Price ($)', fontsize=11)
axes[2].set_title('Scatter Plot: Diamond Prices', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Interpretation
print("\nINTERPRETATION:")
print("-"*80)
print("Visual Observations:")
print("  • Box plot shows numerous outliers above upper whisker")
print("  • Distribution is heavily right-skewed (long tail to the right)")
print("  • Mean ($3,933) > Median ($2,401) confirms right skew")
print("  • Most diamonds are in lower price range")
print("  • High-price outliers represent expensive/rare diamonds")
print("  • Scatter plot shows many points clustered at bottom with outliers above")

print("\n" + "="*80)
print("✅ POINTS: 20/20")

---

## Exercise 2: IQR Method Implementation (25 points)

Implement and apply the IQR method to detect outliers in the `horsepower` variable of the mpg dataset:

**Tasks:**
1. Write a function to detect outliers using IQR method (10 points)
2. Apply with k=1.5 and k=3.0, compare results (8 points)
3. Calculate the impact on mean and median (4 points)
4. Visualize outliers (3 points)

**Function signature:**
```python
def detect_outliers_iqr(data, k=1.5):
    # Your implementation
    return outliers, lower_bound, upper_bound
```

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - Exercise 2

# 1. Function implementation
def detect_outliers_iqr(data, k=1.5):
    """
    Detect outliers using IQR method.
    
    Parameters:
    -----------
    data : pd.Series
        Data to check for outliers
    k : float, default=1.5
        Multiplier for IQR
        
    Returns:
    --------
    outliers : boolean array
        True for outliers, False otherwise
    lower_bound : float
        Lower threshold
    upper_bound : float
        Upper threshold
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    
    outliers = (data < lower_bound) | (data > upper_bound)
    
    return outliers, lower_bound, upper_bound


print("IQR METHOD: Horsepower (MPG Dataset)")
print("="*80)

# Clean data (remove NaN)
hp_clean = mpg['horsepower'].dropna()

# 2. Apply with different k values
print("\nCOMPARISON: k=1.5 vs k=3.0")
print("-"*80)

outliers_15, lower_15, upper_15 = detect_outliers_iqr(hp_clean, k=1.5)
outliers_30, lower_30, upper_30 = detect_outliers_iqr(hp_clean, k=3.0)

print(f"k = 1.5 (Mild outliers):")
print(f"  Bounds: [{lower_15:.2f}, {upper_15:.2f}]")
print(f"  Outliers: {outliers_15.sum()} ({outliers_15.mean()*100:.2f}%)")

print(f"\nk = 3.0 (Extreme outliers):")
print(f"  Bounds: [{lower_30:.2f}, {upper_30:.2f}]")
print(f"  Outliers: {outliers_30.sum()} ({outliers_30.mean()*100:.2f}%)")

print(f"\nDifference: {outliers_15.sum() - outliers_30.sum()} additional outliers with k=1.5")

# 3. Impact on statistics
print("\n3. IMPACT ON MEAN AND MEDIAN")
print("-"*80)

mean_original = hp_clean.mean()
median_original = hp_clean.median()
mean_no_outliers = hp_clean[~outliers_15].mean()
median_no_outliers = hp_clean[~outliers_15].median()

print(f"Original:")
print(f"  Mean: {mean_original:.2f} HP")
print(f"  Median: {median_original:.2f} HP")

print(f"\nWithout outliers (k=1.5):")
print(f"  Mean: {mean_no_outliers:.2f} HP (change: {mean_no_outliers - mean_original:+.2f})")
print(f"  Median: {median_no_outliers:.2f} HP (change: {median_no_outliers - median_original:+.2f})")

print(f"\nImpact:")
if abs(mean_no_outliers - mean_original) > abs(median_no_outliers - median_original):
    print("  Mean is more affected than median (as expected)")
else:
    print("  Median is robust to outliers")

# 4. Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with bounds
axes[0].hist(hp_clean, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(lower_15, color='red', linestyle='--', linewidth=2, label=f'Lower: {lower_15:.1f}')
axes[0].axvline(upper_15, color='red', linestyle='--', linewidth=2, label=f'Upper: {upper_15:.1f}')
axes[0].set_xlabel('Horsepower', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('IQR Bounds (k=1.5)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Scatter with outliers highlighted
colors = ['red' if x else 'blue' for x in outliers_15]
axes[1].scatter(range(len(hp_clean)), hp_clean, c=colors, alpha=0.6)
axes[1].axhline(lower_15, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(upper_15, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Index', fontsize=11)
axes[1].set_ylabel('Horsepower', fontsize=11)
axes[1].set_title('Outliers Highlighted (Red)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ POINTS: 25/25")

---

## Exercise 3: Z-Score vs Modified Z-Score (20 points)

Compare Z-score and Modified Z-score methods on the `carat` variable in diamonds dataset:

**Tasks:**
1. Implement Modified Z-score function (8 points)
2. Apply both methods (threshold=3 for Z, threshold=3.5 for Modified Z) (6 points)
3. Compare number of outliers detected (3 points)
4. Explain which is more appropriate for this data and why (3 points)

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - Exercise 3

# 1. Implement Modified Z-score
def detect_outliers_modified_zscore(data, threshold=3.5):
    """
    Detect outliers using Modified Z-score (robust method).
    
    Parameters:
    -----------
    data : pd.Series
        Data to check for outliers
    threshold : float, default=3.5
        Modified Z-score threshold
        
    Returns:
    --------
    outliers : boolean array
        True for outliers, False otherwise
    modified_z_scores : array
        Modified Z-scores
    """
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    
    # Avoid division by zero
    if mad == 0:
        mad = np.mean(np.abs(data - median))
    
    modified_z_scores = 0.6745 * (data - median) / mad
    outliers = np.abs(modified_z_scores) > threshold
    
    return outliers, modified_z_scores


print("COMPARING Z-SCORE vs MODIFIED Z-SCORE: Carat")
print("="*80)

# 2. Apply both methods
print("\n2. APPLYING BOTH METHODS")
print("-"*80)

# Standard Z-score
z_scores = np.abs(zscore(diamonds['carat']))
outliers_z = z_scores > 3

print(f"Standard Z-Score (threshold=3):")
print(f"  Outliers detected: {outliers_z.sum()} ({outliers_z.mean()*100:.2f}%)")
print(f"  Max Z-score: {z_scores.max():.2f}")

# Modified Z-score
outliers_mz, mod_z_scores = detect_outliers_modified_zscore(diamonds['carat'], threshold=3.5)

print(f"\nModified Z-Score (threshold=3.5):")
print(f"  Outliers detected: {outliers_mz.sum()} ({outliers_mz.mean()*100:.2f}%)")
print(f"  Max Modified Z-score: {np.abs(mod_z_scores).max():.2f}")

# 3. Comparison
print("\n3. COMPARISON")
print("-"*80)

both = outliers_z & outliers_mz
only_z = outliers_z & ~outliers_mz
only_mz = outliers_mz & ~outliers_z

print(f"Detected by both methods: {both.sum()}")
print(f"Only by Z-score: {only_z.sum()}")
print(f"Only by Modified Z-score: {only_mz.sum()}")

# Check data distribution
from scipy.stats import shapiro
stat, p_value = shapiro(diamonds['carat'].sample(5000))  # Sample for speed

print(f"\nNormality Test (Shapiro-Wilk):")
print(f"  p-value: {p_value:.6f}")
print(f"  Normal distribution? {'Yes' if p_value > 0.05 else 'No'}")

# 4. Explanation
print("\n4. WHICH METHOD IS MORE APPROPRIATE?")
print("-"*80)
print("Modified Z-Score is more appropriate for this data because:")
print("  1. Data is NOT normally distributed (p-value < 0.05)")
print("  2. Carat distribution is right-skewed (many small, few large)")
print("  3. Standard Z-score assumes normality")
print("  4. Modified Z-score uses median & MAD (robust to skewness)")
print("  5. Mean and std in standard Z-score are affected by outliers themselves")

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Distribution
axes[0].hist(diamonds['carat'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Carat', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Carat Distribution (Right-Skewed)', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Z-score distribution
axes[1].hist(z_scores, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(3, color='red', linestyle='--', linewidth=2, label='Threshold=3')
axes[1].set_xlabel('Standard Z-Score', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Standard Z-Score Distribution', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# Modified Z-score distribution
axes[2].hist(mod_z_scores, bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[2].axvline(3.5, color='red', linestyle='--', linewidth=2, label='Threshold=3.5')
axes[2].axvline(-3.5, color='red', linestyle='--', linewidth=2)
axes[2].set_xlabel('Modified Z-Score', fontsize=11)
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title('Modified Z-Score Distribution', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ POINTS: 20/20")

---

## Exercise 4: Multivariate Outlier Detection (25 points)

Detect multivariate outliers in the mpg dataset using Isolation Forest:

**Tasks:**
1. Select features: `mpg`, `horsepower`, `weight` (3 points)
2. Apply Isolation Forest with contamination=0.05 (8 points)
3. Visualize multivariate outliers using scatter plots (8 points)
4. Compare with univariate detection on `mpg` alone (6 points)

**Expected:**
- Clean data (remove NaN)
- Proper visualization
- Clear comparison

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - Exercise 4

print("MULTIVARIATE OUTLIER DETECTION: MPG Dataset")
print("="*80)

# 1. Select features and clean data
features = ['mpg', 'horsepower', 'weight']
mpg_clean = mpg[features].dropna()

print(f"\n1. FEATURES SELECTED")
print(f"  Features: {features}")
print(f"  Dataset shape after cleaning: {mpg_clean.shape}")

# 2. Apply Isolation Forest
print("\n2. ISOLATION FOREST")
print("-"*80)

X = mpg_clean.values

iso_forest = IsolationForest(contamination=0.05, random_state=42)
predictions = iso_forest.fit_predict(X)
outliers_multi = predictions == -1
scores = iso_forest.score_samples(X)

print(f"Contamination: 5% (expected outlier rate)")
print(f"Outliers detected: {outliers_multi.sum()} ({outliers_multi.mean()*100:.2f}%)")
print(f"Anomaly scores range: [{scores.min():.3f}, {scores.max():.3f}]")

# Show detected outliers
print(f"\nDetected Multivariate Outliers:")
outlier_df = mpg_clean[outliers_multi].copy()
outlier_df['anomaly_score'] = scores[outliers_multi]
print(outlier_df.sort_values('anomaly_score').head(10))

# 3. Visualize
print("\n3. VISUALIZATION")
print("-"*80)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# MPG vs Horsepower
axes[0, 0].scatter(mpg_clean.loc[~outliers_multi, 'horsepower'], 
                   mpg_clean.loc[~outliers_multi, 'mpg'],
                   alpha=0.6, color='blue', label='Normal', s=50)
axes[0, 0].scatter(mpg_clean.loc[outliers_multi, 'horsepower'], 
                   mpg_clean.loc[outliers_multi, 'mpg'],
                   alpha=0.8, color='red', label='Outliers', s=100, marker='X')
axes[0, 0].set_xlabel('Horsepower', fontsize=11)
axes[0, 0].set_ylabel('MPG', fontsize=11)
axes[0, 0].set_title('MPG vs Horsepower', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# MPG vs Weight
axes[0, 1].scatter(mpg_clean.loc[~outliers_multi, 'weight'], 
                   mpg_clean.loc[~outliers_multi, 'mpg'],
                   alpha=0.6, color='blue', label='Normal', s=50)
axes[0, 1].scatter(mpg_clean.loc[outliers_multi, 'weight'], 
                   mpg_clean.loc[outliers_multi, 'mpg'],
                   alpha=0.8, color='red', label='Outliers', s=100, marker='X')
axes[0, 1].set_xlabel('Weight', fontsize=11)
axes[0, 1].set_ylabel('MPG', fontsize=11)
axes[0, 1].set_title('MPG vs Weight', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Horsepower vs Weight
axes[1, 0].scatter(mpg_clean.loc[~outliers_multi, 'weight'], 
                   mpg_clean.loc[~outliers_multi, 'horsepower'],
                   alpha=0.6, color='blue', label='Normal', s=50)
axes[1, 0].scatter(mpg_clean.loc[outliers_multi, 'weight'], 
                   mpg_clean.loc[outliers_multi, 'horsepower'],
                   alpha=0.8, color='red', label='Outliers', s=100, marker='X')
axes[1, 0].set_xlabel('Weight', fontsize=11)
axes[1, 0].set_ylabel('Horsepower', fontsize=11)
axes[1, 0].set_title('Horsepower vs Weight', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Anomaly scores
scatter = axes[1, 1].scatter(range(len(mpg_clean)), mpg_clean['mpg'], 
                             c=scores, cmap='RdYlGn', alpha=0.6, s=50)
axes[1, 1].set_xlabel('Index', fontsize=11)
axes[1, 1].set_ylabel('MPG', fontsize=11)
axes[1, 1].set_title('Anomaly Scores (Red = More Anomalous)', fontsize=12, fontweight='bold')
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_label('Anomaly Score', fontsize=10)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Compare with univariate detection
print("\n4. COMPARISON WITH UNIVARIATE DETECTION")
print("-"*80)

# Univariate detection on MPG only
outliers_univar, _, _ = detect_outliers_iqr(mpg_clean['mpg'], k=1.5)

print(f"Univariate (MPG only - IQR): {outliers_univar.sum()} outliers")
print(f"Multivariate (MPG + HP + Weight - IF): {outliers_multi.sum()} outliers")

# Overlap analysis
both = outliers_univar & outliers_multi
only_univar = outliers_univar & ~outliers_multi
only_multi = outliers_multi & ~outliers_univar

print(f"\nOverlap Analysis:")
print(f"  Detected by both: {both.sum()}")
print(f"  Only univariate: {only_univar.sum()}")
print(f"  Only multivariate: {only_multi.sum()}")

print(f"\nKey Insight:")
print(f"  Multivariate detection found {only_multi.sum()} outliers that univariate missed.")
print(f"  These are cases where MPG alone looks normal, but the combination")
print(f"  of MPG + Horsepower + Weight is unusual.")

print("\n" + "="*80)
print("✅ POINTS: 25/25")

---

## Exercise 5: Outlier Treatment Comparison (20 points)

Compare different treatment strategies for the `price` variable in diamonds dataset:

**Tasks:**
1. Detect outliers using IQR method (k=1.5) (4 points)
2. Apply THREE treatments: deletion, capping (5th-95th percentile), log transformation (9 points)
3. Compare mean and standard deviation for all strategies (4 points)
4. Recommend best treatment with justification (3 points)

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - Exercise 5

print("OUTLIER TREATMENT COMPARISON: Diamond Prices")
print("="*80)

# 1. Detect outliers
print("\n1. OUTLIER DETECTION (IQR, k=1.5)")
print("-"*80)

outliers, lower, upper = detect_outliers_iqr(diamonds['price'], k=1.5)

print(f"Bounds: [${lower:,.2f}, ${upper:,.2f}]")
print(f"Outliers detected: {outliers.sum()} ({outliers.mean()*100:.2f}%)")

# 2. Apply treatments
print("\n2. APPLYING TREATMENTS")
print("-"*80)

# Strategy 1: Deletion
diamonds_deleted = diamonds[~outliers].copy()
print(f"\nStrategy 1: DELETION")
print(f"  Removed {outliers.sum()} outliers")
print(f"  Remaining: {len(diamonds_deleted)} rows")

# Strategy 2: Capping
lower_cap = diamonds['price'].quantile(0.05)
upper_cap = diamonds['price'].quantile(0.95)

diamonds_capped = diamonds.copy()
diamonds_capped['price_capped'] = diamonds_capped['price'].clip(lower=lower_cap, upper=upper_cap)

capped_count = (diamonds['price'] != diamonds_capped['price_capped']).sum()
print(f"\nStrategy 2: CAPPING (5th-95th percentile)")
print(f"  Lower cap: ${lower_cap:,.2f}")
print(f"  Upper cap: ${upper_cap:,.2f}")
print(f"  Values capped: {capped_count}")

# Strategy 3: Log Transformation
diamonds_log = diamonds.copy()
diamonds_log['price_log'] = np.log1p(diamonds_log['price'])

print(f"\nStrategy 3: LOG TRANSFORMATION")
print(f"  Applied: log1p(price)")
print(f"  All values transformed")

# 3. Compare statistics
print("\n3. COMPARISON OF STATISTICS")
print("-"*80)

strategies = {
    'Original': diamonds['price'],
    'Deleted': diamonds_deleted['price'],
    'Capped': diamonds_capped['price_capped'],
    'Log (back-transformed)': np.expm1(diamonds_log['price_log'])
}

print(f"\n{'Strategy':<25} {'Mean':>12} {'Std':>12} {'Median':>12} {'N':>8}")
print("-"*75)
for name, data in strategies.items():
    print(f"{name:<25} ${data.mean():>11,.0f} ${data.std():>11,.0f} ${data.median():>11,.0f} {len(data):>8,}")

# Calculate changes
print(f"\nCHANGES FROM ORIGINAL:")
print("-"*75)
original_mean = diamonds['price'].mean()
original_std = diamonds['price'].std()

for name, data in strategies.items():
    if name != 'Original':
        mean_change = data.mean() - original_mean
        std_change = data.std() - original_std
        print(f"{name:<25} Mean: ${mean_change:>+10,.0f}  Std: ${std_change:>+10,.0f}")

# 4. Recommendation
print("\n4. RECOMMENDATION")
print("="*80)

print("\nBest Treatment: CAPPING")
print("\nJustification:")
print("  1. Preserves sample size (critical with 53,940 observations)")
print("  2. Reduces extreme influence without data loss")
print("  3. Mean reduced moderately (-$186), more robust")
print("  4. Standard deviation reduced significantly (-$719)")
print("  5. Suitable for regression modeling (reduces leverage)")
print("  6. Interpretability maintained (still in original units)")
print("\nWhen to use alternatives:")
print("  • Deletion: If outliers are errors (< 1% of data)")
print("  • Log Transform: For multiplicative relationships, need normality")
print("  • Keep original: For descriptive analysis, no modeling")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original
axes[0, 0].hist(diamonds['price'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Price ($)', fontsize=10)
axes[0, 0].set_ylabel('Frequency', fontsize=10)
axes[0, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

# Deleted
axes[0, 1].hist(diamonds_deleted['price'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_xlabel('Price ($)', fontsize=10)
axes[0, 1].set_ylabel('Frequency', fontsize=10)
axes[0, 1].set_title('After Deletion', fontsize=11, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# Capped
axes[1, 0].hist(diamonds_capped['price_capped'], bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 0].set_xlabel('Price ($)', fontsize=10)
axes[1, 0].set_ylabel('Frequency', fontsize=10)
axes[1, 0].set_title('After Capping', fontsize=11, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Log transformed
axes[1, 1].hist(diamonds_log['price_log'], bins=50, edgecolor='black', alpha=0.7, color='plum')
axes[1, 1].set_xlabel('Log(Price)', fontsize=10)
axes[1, 1].set_ylabel('Frequency', fontsize=10)
axes[1, 1].set_title('Log Transformed', fontsize=11, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ POINTS: 20/20")

---

## BONUS Exercise: Complete Outlier Pipeline (10 points)

Build a complete automated outlier detection and treatment pipeline:

**Tasks:**
1. Create a function that takes data, column, and detection method (5 points)
2. Function should detect outliers using chosen method (3 points)
3. Report statistics and visualize (2 points)

**Function signature:**
```python
def outlier_pipeline(data, column, method='iqr', treatment='report'):
    # Your implementation
    return results_dict
```

**Test on:** `mpg` column in mpg dataset

### Your Solution:

In [ ]:
# YOUR CODE HERE








### Solution:

In [ ]:
# SOLUTION - BONUS Exercise

def outlier_pipeline(data, column, method='iqr', treatment='report'):
    """
    Complete outlier detection and treatment pipeline.
    
    Parameters:
    -----------
    data : DataFrame
        Input data
    column : str
        Column to analyze
    method : str
        Detection method: 'iqr', 'zscore', 'iforest'
    treatment : str
        Treatment: 'report', 'delete', 'cap'
        
    Returns:
    --------
    results : dict
        Analysis results
    """
    results = {}
    
    print("="*80)
    print(f"OUTLIER DETECTION PIPELINE: {column}")
    print(f"Method: {method.upper()} | Treatment: {treatment.upper()}")
    print("="*80)
    
    # Clean data
    clean_data = data[column].dropna()
    
    # 1. Detection
    print("\n1. OUTLIER DETECTION")
    print("-"*80)
    
    if method == 'iqr':
        outliers, lower, upper = detect_outliers_iqr(clean_data, k=1.5)
        print(f"Method: IQR (k=1.5)")
        print(f"Bounds: [{lower:.2f}, {upper:.2f}]")
    
    elif method == 'zscore':
        z_scores = np.abs(zscore(clean_data))
        outliers = z_scores > 3
        print(f"Method: Z-Score (threshold=3)")
        print(f"Max Z-score: {z_scores.max():.2f}")
    
    elif method == 'iforest':
        iso_forest = IsolationForest(contamination=0.05, random_state=42)
        predictions = iso_forest.fit_predict(clean_data.values.reshape(-1, 1))
        outliers = predictions == -1
        print(f"Method: Isolation Forest (contamination=5%)")
    
    print(f"Outliers detected: {outliers.sum()} ({outliers.mean()*100:.2f}%)")
    results['outliers_count'] = outliers.sum()
    results['outliers_pct'] = outliers.mean() * 100
    
    # 2. Statistics
    print("\n2. IMPACT ASSESSMENT")
    print("-"*80)
    
    mean_with = clean_data.mean()
    mean_without = clean_data[~outliers].mean()
    
    print(f"Mean (with outliers): {mean_with:.2f}")
    print(f"Mean (without outliers): {mean_without:.2f}")
    print(f"Change: {mean_without - mean_with:+.2f}")
    
    results['mean_change'] = mean_without - mean_with
    
    # 3. Visualization
    print("\n3. VISUALIZATION")
    print("-"*80)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Box plot
    bp = axes[0].boxplot(clean_data, vert=True, patch_artist=True)
    axes[0].set_ylabel(column, fontsize=11)
    axes[0].set_title('Box Plot', fontsize=12, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Histogram
    axes[1].hist(clean_data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].set_xlabel(column, fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].set_title('Distribution', fontsize=12, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Scatter with outliers
    colors = ['red' if x else 'blue' for x in outliers]
    axes[2].scatter(range(len(clean_data)), clean_data, c=colors, alpha=0.6)
    axes[2].set_xlabel('Index', fontsize=11)
    axes[2].set_ylabel(column, fontsize=11)
    axes[2].set_title('Outliers Highlighted', fontsize=12, fontweight='bold')
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 4. Treatment
    print("\n4. TREATMENT")
    print("-"*80)
    
    if treatment == 'delete':
        treated_data = data[~outliers].copy()
        print(f"Deleted {outliers.sum()} outliers")
        results['data'] = treated_data
    
    elif treatment == 'cap':
        lower_cap = clean_data.quantile(0.05)
        upper_cap = clean_data.quantile(0.95)
        treated_data = data.copy()
        treated_data[column + '_treated'] = treated_data[column].clip(lower=lower_cap, upper=upper_cap)
        print(f"Capped at [{lower_cap:.2f}, {upper_cap:.2f}]")
        results['data'] = treated_data
    
    else:
        print("Report only - no treatment applied")
        results['data'] = data
    
    print("\n" + "="*80)
    print("✅ PIPELINE COMPLETE")
    print("="*80)
    
    return results


# Test the pipeline
print("TESTING OUTLIER PIPELINE")
print("="*80)

results = outlier_pipeline(mpg, 'mpg', method='iqr', treatment='report')

print(f"\nRESULTS:")
print(f"  Outliers found: {results['outliers_count']}")
print(f"  Percentage: {results['outliers_pct']:.2f}%")
print(f"  Mean change (if removed): {results['mean_change']:.2f}")

print("\n" + "="*80)
print("✅ BONUS POINTS: 10/10")

---

## Grading Summary

Run the cell below to see your total score:

In [ ]:
# Grading (Fill in your scores)
scores = {
    'Exercise 1 - Visual Detection': 20,           # Max: 20
    'Exercise 2 - IQR Method': 25,                 # Max: 25
    'Exercise 3 - Z-Score Comparison': 20,         # Max: 20
    'Exercise 4 - Multivariate Detection': 25,     # Max: 25
    'Exercise 5 - Treatment Comparison': 20,       # Max: 20
    'BONUS - Complete Pipeline': 10                # Max: 10
}

total_score = sum(scores.values())
max_score = 110

print("="*80)
print("EXERCISE GRADING SUMMARY")
print("="*80)

for exercise, score in scores.items():
    print(f"{exercise:45s}: {score:3d} points")

print("-"*80)
print(f"{'TOTAL SCORE':45s}: {total_score:3d}/{max_score}")
print("="*80)

# Grade
if total_score >= 90:
    grade = "Excellent ✅"
    emoji = "🎉"
elif total_score >= 70:
    grade = "Good 👍"
    emoji = "👏"
elif total_score >= 50:
    grade = "Needs Improvement ⚠️"
    emoji = "📚"
else:
    grade = "Review Material 📚"
    emoji = "🔄"

print(f"\nGrade: {grade} {emoji}")
print(f"Percentage: {total_score/max_score*100:.1f}%")

if total_score >= 90:
    print("\n🌟 Outstanding! You've mastered outlier detection and treatment!")
elif total_score >= 70:
    print("\n💪 Great work! Keep practicing advanced methods!")
else:
    print("\n📖 Review theory and practical notebooks for better understanding.")

---

## Key Learnings

After completing these exercises, you should be able to:

✅ Visualize outliers using box plots, histograms, and scatter plots  
✅ Apply IQR method for robust outlier detection  
✅ Use Z-score and Modified Z-score methods appropriately  
✅ Implement advanced methods (Isolation Forest, LOF)  
✅ Detect multivariate outliers across multiple variables  
✅ Compare different treatment strategies  
✅ Make informed decisions about outlier treatment  
✅ Build automated outlier detection pipelines

## Decision Guidelines

**When to use each method:**
- **IQR:** Skewed data, initial screening, robust detection
- **Z-Score:** Normal distributions, comparative analysis
- **Modified Z-Score:** Non-normal data, robust to outliers
- **Isolation Forest:** Multivariate, high-dimensional data
- **LOF:** Varying density, local anomalies

**Treatment selection:**
- **Delete:** Data errors (< 1% of data)
- **Cap:** Modeling, reduce extreme influence
- **Transform:** Normalization, reduce skewness
- **Keep:** Descriptive analysis, valid rare events

## Next Steps

1. Practice with your own datasets
2. Experiment with different methods
3. Build domain-specific pipelines
4. Document your decisions
5. Apply to real-world problems

**Congratulations on completing Unit 6! 🎊**